In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# שלום עולם: המעגל הקוונטי הראשון שלך

בנה [מצב Bell](https://en.wikipedia.org/wiki/Bell_state) (שני קיוביטים שזורים) והפעל אותו בשלוש דרכים:

1. **סימולציה אידיאלית** — תוצאות מושלמות, ללא צורך בחשבון
2. **סימולציה עם רעש** — מדמה מכשיר אמיתי, ללא צורך בחשבון
3. **חומרה קוונטית אמיתית** — דורש [חשבון IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## בניית המעגל

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## אפשרות 1: סימולציה אידיאלית (ללא צורך בחשבון)
משתמש ב-`StatevectorSampler` — סימולטור מקומי עם תוצאות מושלמות, ללא רעש.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## אפשרות 2: סימולציה עם רעש (ללא צורך בחשבון)
משתמש ב-`FakeManilaV2` — סימולטור מקומי המחקה מכשיר IBM Quantum אמיתי, כולל מאפייני הרעש שלו. המעגל חייב להיות מותאם (transpiled) תחילה לסט השערים וחיבוריות הקיוביטים של המכשיר.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## אפשרות 3: חומרה קוונטית אמיתית
דורש חשבון IBM Quantum. ראה [הגדרת חשבון IBM Quantum שלך](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) לפרטים.

אם עדיין לא שמרת את האישורים שלך בסשן Binder זה, הפעל זאת תחילה:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**הערה:** משימות על חומרה אמיתית עשויות לקחת זמן בהתאם לזמני התור. אם התא עדיין פועל, תוכל לבדוק את סטטוס המשימה שלך ולראות את התוצאות ב-[quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## מה הלאה?
- **[מדריכים](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — מדריכים שלב-אחר-שלב על אלגוריתמים, הפחתת שגיאות, transpilation ועוד
- **[קורסים](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — מסלולי לימוד מובנים מיסודות הקוונטום לחישוב בקנה מידה שימושי
- **[מצב בדיקה מקומי](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — הפעל את רוב ה-notebooks ללא חשבון IBM Quantum